# Emerging Technologies

This notebook contains solutions for the assessment problems on classical and quantum algorithms using the Deutsch–Jozsa algorithm.

## Import Required

In [13]:
import random # problem 1 - random is used for uniform random choices and shuffling;
import numpy as np #  problem 1 - numpy (np) is used later for numeric checks and array operations; 
import itertools as it # problem 1 - itertools (it) is used for generating all binary tuples.


## Introduction

### Classical vs Quantum Information

**[Classical Information](https://en.wikipedia.org/wiki/Classical_information_theory)** - Think about how a CD or DVD stores music and videos. The disc surface contains tiny physical pits — a pit represents a 0, and a flat area (called a "land") represents a 1. When you play the disc, a laser reads these pits one by one, [decoding the binary data sequentially.](https://www.sciencedirect.com/topics/engineering/compact-disc) This is how classical computers work: they process information as definite 0s and 1s, checking each value individually. If you wanted to find out whether a mystery function always gives the same answer (constant) or gives a mix of answers (balanced), you would have to test many inputs one at a time — potentially needing up to 9 queries for a 4-input function.

**[Quantum Information](https://en.wikipedia.org/wiki/Quantum_information)** - Now imagine you need to decide whether to bring a raincoat before going outside. Classically, you would check the weather forecast first, then decide. But what if you could somehow be prepared for all weather possibilities at once? This is the essence of [quantum superposition](https://en.wikipedia.org/wiki/Quantum_superposition) — a **[quantum bit (qubit)](https://en.wikipedia.org/wiki/Qubit)** can exist as both 0 AND 1 simultaneously, like being in a state of "maybe rain, maybe sunshine" until you observe it. The [Deutsch–Jozsa algorithm](https://arxiv.org/abs/quant-ph/9708016) exploits this property: by putting qubits into superposition, we can query a function with all possible inputs at once, and through **[quantum interference](https://en.wikipedia.org/wiki/Wave_interference#Quantum_interference)**, the answer (constant or balanced) reveals itself in just a single query. This is the **[quantum advantage](https://en.wikipedia.org/wiki/Quantum_supremacy)** we will explore in this notebook.


## Problem 1: Generating Random Boolean Functions

**Instructions:**  
Write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of constant or balanced functions taking four **[Boolean](https://en.wikipedia.org/wiki/Boolean_data_type)** inputs.

### 1. Classical Systems and Their State Sets

A classical system is defined by the set of states it can be in.

Examples:

- If \(X\) is a **bit**, then  
Σ = \{0,1\}

- If \(X\) is a **six-sided die**, then  
Σ = \{1,2,3,4,5,6\}


- If \(X\) is a **fan switch**, then  
Σ = {high, medium, low, off}

The physical representation does not matter — only the distinct states matter.

“A bit is just a system that can be in two different states.”

### 2. When We Know the State Exactly

Sometimes we know the state with certainty.

Example:  
If the fan switch is set to *high*, we know the exact classical state.

But in real computation (e.g., networking), we often **don’t** know the state.

### 3. When We Don’t Know the State: Probabilistic States

“If you do any network programming, you’re going to take bits from the network, and you don’t know what they are.”

Real-world information is often uncertain. You cannot predict incoming bits. If you could predict them, the channel would carry less information (because information = unpredictability).

Suppose we believe:

The probability of \(X=0\) is $ \Pr(X=0) = \frac{3}{4} $, and the probability of $X=1$ is $ \Pr(X=1) = \frac{1}{4} $.

This is exactly the IBM Quantum textbook example.

### 4. Representing Uncertainty with Probability Vectors

We represent the probabilistic state as a **[column vector](https://en.wikipedia.org/wiki/Row_and_column_vectors)**:

\[
\begin{bmatrix}
0.75 \\
0.25
\end{bmatrix}
\]

- Top entry = probability of 0  
- Bottom entry = probability of 1  
- They must sum to 1  

A **[probability vector](https://en.wikipedia.org/wiki/Probability_vector)** must satisfy:

1. All entries ≥ 0  
2. Entries sum to 1  

This is the classical analogue of **[quantum state vectors](https://en.wikipedia.org/wiki/Quantum_state#Pure_states_as_rays_in_a_Hilbert_space)**.

### 5. Special Probability Vectors: The Basis States

Two special vectors represent definite classical states:

$$
|0\rangle = \begin{bmatrix}1 \\ 0\end{bmatrix}, \qquad
|1\rangle = \begin{bmatrix}0 \\ 1\end{bmatrix}.
$$

These are the **basis vectors** of classical information. Any probability vector can be written as:

$$
p(0)\,|0\rangle + p(1)\,|1\rangle.
$$

### 6. Demonstrating Probability Vectors in Python




In [14]:
# Example probability vector
p = np.array([[0.75],
              [0.25]])
# Check that the probabilities sum to 1
p.sum()


np.float64(1.0)

### 7. Classical Functions on a Single Bit

A function that takes one bit in and outputs one bit can only behave in **four** possible ways:

| Name | Description | Mapping |
|------|-------------|---------|
| F1 | Constant 0 | 0→0, 1→0 |
| F2 | Identity | 0→0, 1→1 |
| F3 | NOT | 0→1, 1→0 |
| F4 | Constant 1 | 0→1, 1→1 |

“No matter how complicated thecode is, at the end it’s one of these four.”

### 8. Implementing the Four Functions in Python

In [15]:
def f1(a):
    return 0

def f2(a):
    return a

def f3(a):
    return 1 - a

def f4(a):
    return 1

functions = [f1, f2, f3, f4]

### 9. Choosing a Random Function

In [16]:
f = random.choice(functions) 
f

<function __main__.f3(a)>

### 10. Identifying the Random Function

“The only thing can do is ask the function what happens if put in 0 and what happens if put in 1.”

In [17]:
f(0), f(1)

(1, 0)

### 11. Determining Which Function It Is

We compare the outputs to the four possibilities.

In [18]:
def identify_function(f):
    out0 = f(0)
    out1 = f(1)
    
    if out0 == 0 and out1 == 0:
        return "F1 (constant 0)"
    if out0 == 0 and out1 == 1:
        return "F2 (identity)"
    if out0 == 1 and out1 == 0:
        return "F3 (NOT)"
    if out0 == 1 and out1 == 1:
        return "F4 (constant 1)"

identify_function(f)

'F3 (NOT)'

### 12. Why This Matters for Quantum Computing

Classically, we need **two queries** to fully identify the function.

Quantum computing asks:

> Can we learn something about the function with **one** query?

This leads directly to **[Deutsch's algorithm](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm#Deutsch's_algorithm)**, the first example of **[quantum speedup](https://en.wikipedia.org/wiki/Quantum_computing#Potential)**.

---

### Random Functions (classical toy problem)

One of the simplest toy problems for introducing quantum speedups is to work with raw bits.  
This notebook sets up functions $f:\{0,1\}^n \to \{0,1\}$ that are guaranteed to be either **constant** or **balanced**.  
The $n=3$ example, the random-tuple representation, the combinatorics for balanced functions, and Python code to generate and test random functions are included.

#### Purpose

The problem is reduced to its barest form: functions on bits. No semantic meaning is attached to inputs or outputs — they are raw bits.  
The task: given **[oracle](https://en.wikipedia.org/wiki/Oracle_machine)** access to a function $f$ that is either constant or balanced, determine which type it is. The classical baseline requires many queries; this sets up the Deutsch / Deutsch–Jozsa style question.

#### Definitions

- **[Constant function](https://en.wikipedia.org/wiki/Constant_function):** the output is identical for every input. For a length-$2^n$ tuple the output is either all zeros or all ones.  
- **[Balanced function](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm#Problem_statement):** exactly half of the $2^n$ outputs are $0$ and half are $1$ (so there are $2^{n-1}$ zeros and $2^{n-1}$ ones).

#### Example: $n = 3$ (table representation)

All input triples are listed and one possible function output column is shown. The function column is a length-$2^n$ tuple.

| **a** | **b** | **c** | **f(a,b,c)** |
|-----:|-----:|-----:|:------------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |

- **Constant examples:** $(0,0,0,0,0,0,0,0)$ and $(1,1,1,1,1,1,1,1)$.  
- **Balanced examples:** any tuple with exactly four $0$'s and four $1$'s, e.g. $(0,0,0,0,1,1,1,1)$ or $(1,0,0,1,1,0,0,1)$.

#### Combinatorics for balanced functions ($n=3$)

A balanced function for $n=3$ must have exactly half the outputs $0$ and half $1$, so four $0$'s and four $1$'s in the length-$8$ tuple.  
The number of distinct balanced tuples equals the number of ways to choose which $4$ of the $8$ positions are ones:

$$
\binom{8}{4}
$$

(See **[binomial coefficient](https://en.wikipedia.org/wiki/Binomial_coefficient)**)

This can be computed by counting ordered selections then removing order:

- Ordered selections of 4 distinct positions: $8 \times 7 \times 6 \times 5 = 1680$.
- Divide by the number of orders of those 4 positions, $4! = 24$.

Thus

$$
\binom{8}{4} = \frac{8\cdot7\cdot6\cdot5}{4!} = \frac{1680}{24} = 70.
$$

Therefore there are **70** distinct balanced functions when $n=3$.

#### Random-function model and probabilities

The random model used here is:

1. Choose uniformly between the two types $\{\text{constant},\text{balanced}\}$ (50/50).  
2. If constant: choose uniformly between the two constant tuples (all-0 or all-1).  
3. If balanced: choose uniformly among all $\binom{2^n}{2^{n-1}}$ balanced tuples.

For $n=3$:

- Probability of selecting a particular constant tuple (e.g., all zeros):
  $$
  P(\text{specific constant}) = \tfrac{1}{2}\times\tfrac{1}{2} = \tfrac{1}{4}.
  $$
- Probability of selecting a particular balanced tuple:
  $$
  P(\text{specific balanced}) = \tfrac{1}{2}\times\tfrac{1}{70} = \tfrac{1}{140}.
  $$

---
#### One-bit examples

**Reference:** These represent all possible **[Boolean functions](https://en.wikipedia.org/wiki/Boolean_function)** of one variable. The complete **[truth table](https://en.wikipedia.org/wiki/Truth_table)** enumeration is a fundamental concept in digital logic and the basis for understanding Deutsch's algorithm (see [IBM Quantum Learning: Deutsch-Jozsa Algorithm](https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/quantum-query-algorithms#the-deutsch-jozsa-algorithm)).

1. Function definitions: four minimal one-bit functions are defined to illustrate constant and non-constant behaviours: f1 and f4 are constant, f2 is identity, f3 is logical NOT.

In [ ]:
# One-bit example functions (illustration)
def f1(a):
    return 0 # constant function that always returns 0

def f2(a):
    return a # identity function: returns the input bit unchanged

def f3(a):
    return 0 if a else 1 # NOT function: returns 1 when input is 0, and 0 when input is 1

def f4(a):
    return 1 # constant function that always returns 1


2. random_one_bit_function: demonstrates that functions are **[first-class objects](https://en.wikipedia.org/wiki/First-class_function)** and can be stored in lists; **[random.choice](https://docs.python.org/3/library/random.html#random.choice)** returns one function object using **[uniform random selection](https://en.wikipedia.org/wiki/Discrete_uniform_distribution)**.
3. Test: the chosen function is evaluated on both inputs 0 and 1 to show its behaviour.

In [32]:
# Example: random one-bit function
def random_one_bit_function():
    fns = [f1, f2, f3, f4]  # list of candidate one-bit functions
    return random.choice(fns)  # select and return one function uniformly at random

# Quick test
f = random_one_bit_function()  # obtain a random one-bit function
print("f(0), f(1) =", f(0), f(1))  # evaluate the chosen function on both possible inputs

f(0), f(1) = 1 0


---
#### random_tuple generator
**Purpose**  
Produce a length-$2^n$ tuple representing either a **constant** function (all entries identical) or a **balanced** function (exactly half zeros and half ones).

**Reference:** This implements a **[uniform sampling](https://en.wikipedia.org/wiki/Discrete_uniform_distribution)** strategy over the space of promise functions used in the Deutsch-Jozsa problem. The **[Fisher-Yates shuffle](https://en.wikipedia.org/wiki/Fisher%E2%80%93Yates_shuffle)** (implemented by `random.shuffle`) ensures unbiased random permutation of balanced functions.

* Type selection: ftype is chosen uniformly between 'constant' and 'balanced'.
* Constant branch: zero_or_one picks 0 or 1; (zero_or_one,) * (2**n) constructs the repeated-tuple efficiently.
* Balanced branch: a list is built with the required counts of zeros and ones; random.shuffle randomizes the order; conversion to tuple makes the result hashable/ immutable and consistent with the constant case.
* Notes: using a list for shuffling is necessary because random. shuffle operates in-place on mutable sequences.



In [ ]:
def random_tuple(n):
    """Return a random constant or balanced tuple of length 2**n."""
    ftype = random.choice(['constant', 'balanced'])  # choose function type uniformly

    if ftype == 'constant':
        zero_or_one = random.choice([0, 1]) # choose whether the constant is 0 or 1
        t = (zero_or_one,) * (2**n) # create a tuple repeating that bit 2**n times

    else:
        # balanced: create a list with half zeros and half ones, then shuffle
        t = [0] * (2**(n-1)) + [1] * (2**(n-1)) # list with 2^(n-1) zeros followed by 2^(n-1) ones
        random.shuffle(t) # shuffle in-place to randomize positions
        t = tuple(t) # convert list to tuple for immutability/consistency

    return t # return the resulting tuple


---
#### Helpers bin_args_to_int and closure fclosure

**Purpose**  
- `bin_args_to_int` converts a binary argument list into an integer index for tuple lookup.  
- `fclosure` creates a **[closure](https://en.wikipedia.org/wiki/Closure_(computer_programming))** that implements a fixed random oracle (constant or balanced) on $n$ bits.

**Reference:** The encoding technique converts **[binary numbers](https://en.wikipedia.org/wiki/Binary_number)** to **[decimal indices](https://en.wikipedia.org/wiki/Positional_notation)** for array lookup—a standard indexing strategy. The closure pattern captures the random tuple in its **[lexical scope](https://en.wikipedia.org/wiki/Scope_(computer_science)#Lexical_scope)**, modeling the "black box" oracle paradigm central to **[query complexity theory](https://en.wikipedia.org/wiki/Query_complexity)** and quantum algorithms (see [Quantum Algorithm Zoo](https://quantumalgorithmzoo.org/)).


* Closure pattern: f_a is fixed when fclosure is called; the returned f uses that fixed tuple, modelling an oracle.* Indexing convention: the first argument corresponds to the most-significant bit in the binary string; maintain this ordering consistently.

In [ ]:
def bin_args_to_int(*args):
    """Convert binary args (truthy -> '1', falsy -> '0') to integer index."""
    bits = ''.join('1' if i else '0' for i in args) # build a binary string from positional args
    return int(bits, 2) # parse the binary string as base-2 integer

def fclosure(n):
    """Return a closure f(*args) that implements a random constant or balanced function on n bits."""
    f_a = random_tuple(n)                            # generate the underlying length-2^n tuple once

    def f(*args):
        if len(args) != n:
            return None  # guard: require exactly n input bits
        return f_a[bin_args_to_int(*args)]# map the input bits to an index and return the tuple element

    return f # return the inner function (closure)


---
#### try_all and is_constant_or_balanced

**Purpose**  
- `try_all` exhaustively evaluates the oracle on all $2^n$ inputs (classical brute-force).  
- `is_constant_or_balanced` classifies the oracle as `'constant'`, `'balanced'`, or `'neither'` using the exhaustive outputs.

**Reference:** This implements **[brute-force search](https://en.wikipedia.org/wiki/Brute-force_search)** over the input space—the classical baseline requiring $O(2^n)$ queries in the worst case. The **[itertools.product](https://docs.python.org/3/library/itertools.html#itertools.product)** generates the **[Cartesian product](https://en.wikipedia.org/wiki/Cartesian_product)** $(\{0,1\}^n)$ of all binary inputs. This exponential cost is what the Deutsch-Jozsa algorithm reduces to a single quantum query (see [Nielsen & Chuang, "Quantum Computation and Quantum Information", Section 1.4.3](https://en.wikipedia.org/wiki/Quantum_Computation_and_Quantum_Information)).

In [ ]:
def try_all(f, n):
    """Try all 2**n binary inputs on f and return the list of outputs."""
    bin_tuples_n = it.product((0,1), repeat=n)  # generator for all binary tuples of length n
    vals = [f(*t) for t in bin_tuples_n] # evaluate f on each tuple and collect outputs
    return vals

def is_constant_or_balanced(f, n):
    """Return 'constant', 'balanced', or 'neither' for function f on n bits (naive test)."""
    results = np.array(try_all(f, n)) # get outputs and convert to numpy array for numeric checks

    # Check if all outputs are 0 or all outputs are 1
    if np.all(results == 0) or np.all(results == 1):
        return 'constant'

    # Check if the number of ones equals 2^(n-1) (balanced)
    elif results.sum() == 2**(n-1):
        return 'balanced'

    # Otherwise the function is neither constant nor balanced (should not occur under the promise model)
    else:
        return 'neither'

# Quick demonstration
f_test = fclosure(3) # create a random function on 3 bits
vals = try_all(f_test, 3) # enumerate outputs for all 8 inputs
print("outputs:", vals) # print the full output tuple
print("classification:", is_constant_or_balanced(f_test, 3))  # print the classification result


outputs: [0, 0, 0, 0, 0, 0, 0, 0]
classification: constant


In [54]:
# Problem 1 solution goes here
# Deterministic operations
def random_constant_balanced():
    """
    Returns a randomly chosen constant or balanced function
    that takes 4 Boolean inputs.
    """
    # Step 1: Randomly decide constant or balanced
    is_constant = random.choice([True, False])
    
    if is_constant:
        # Step 2a: Constant function - all 16 outputs are the same
        value = random.choice([0, 1])
        outputs = [value] * 16
    else:
        # Step 2b: Balanced function - 8 zeros and 8 ones, shuffled
        outputs = [0] * 8 + [1] * 8
        random.shuffle(outputs)
    
    # Step 3: Return a function that uses this lookup table
    def f(a, b, c, d):
        # Convert 4 bits to index (0-15)
        index = 8*a + 4*b + 2*c + d
        return outputs[index]
    
    return f

In [55]:
# Test the function
f = random_constant_balanced()

# Print all outputs to see the pattern
print("Input -> Output")
for i in range(16):
    a = (i >> 3) & 1
    b = (i >> 2) & 1
    c = (i >> 1) & 1
    d = i & 1
    print(f"({a},{b},{c},{d}) -> {f(a,b,c,d)}")

# Count outputs to verify constant/balanced
outputs = [f((i>>3)&1, (i>>2)&1, (i>>1)&1, i&1) for i in range(16)]
zeros = outputs.count(0)
ones = outputs.count(1)
print(f"\nZeros: {zeros}, Ones: {ones}")
print(f"Type: {'Constant' if zeros == 0 or zeros == 16 else 'Balanced'}")

Input -> Output
(0,0,0,0) -> 0
(0,0,0,1) -> 1
(0,0,1,0) -> 0
(0,0,1,1) -> 1
(0,1,0,0) -> 0
(0,1,0,1) -> 0
(0,1,1,0) -> 1
(0,1,1,1) -> 0
(1,0,0,0) -> 1
(1,0,0,1) -> 1
(1,0,1,0) -> 0
(1,0,1,1) -> 0
(1,1,0,0) -> 0
(1,1,0,1) -> 1
(1,1,1,0) -> 1
(1,1,1,1) -> 1

Zeros: 8, Ones: 8
Type: Balanced


## Problem 2: Classical Testing for Function Type

**Instructions:**  
Write a Python function `determine_constant_balanced(f)` that analyzes a function f and returns "constant" or "balanced". Include a brief note on efficiency.

In [56]:
# Problem 2 solution goes here

## Problem 3: Quantum Oracles

**Instructions:**  
Using **[Qiskit](https://qiskit.org/)**, create **[quantum oracles](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm#Algorithm)** for each of the possible single-Boolean-input functions in Deutsch's algorithm. Explain how each oracle works.

In [57]:
# Problem 3 solution goes here

## Problem 4: Deutsch's Algorithm with Qiskit

**Instructions:**  
Design a **[quantum circuit](https://en.wikipedia.org/wiki/Quantum_circuit)** that solves Deutsch's problem for single-input functions. Demonstrate its use with each oracle and explain the interference pattern.

In [58]:
# Problem 4 solution goes here

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm

**Instructions:**  
Use Qiskit to create a quantum circuit for four-bit functions generated in Problem 1. Demonstrate it on constant and balanced functions and explain the results.

In [59]:
# Problem 5 solution goes here